# Machine Learning Pipelines

Various classifiers from PySpark's ml library were selected as candidates to fit the data concerning customer churn since the data pipeline was already hosted on PySpark, and the customer churn problem appears to be a binary classification problem. PySpark appears to be convenient to use in this case, so the plan is to build ml models in PySpark to fit and predict customer churn.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructField
    , StringType
    , IntegerType
    , DoubleType
    , BooleanType
    , StructType
)
from pyspark.ml.feature import (
    VectorAssembler
    , OneHotEncoder
    , StringIndexer
    , Imputer
)
from pyspark.ml.classification import (
    LogisticRegression
    , DecisionTreeClassifier
    , GBTClassifier
    , RandomForestClassifier
)
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import (
    BinaryClassificationEvaluator
    , MulticlassClassificationEvaluator
)

In [0]:
spark = SparkSession.builder.appName('ML_Pipeline').getOrCreate()

In [0]:
my_schema = StructType(fields=[
    StructField('customer_id', IntegerType(), True)
    , StructField('surname', StringType(), True)
    , StructField('credit_score', IntegerType(), True)
    , StructField('geography', StringType(), True)
    , StructField('gender', StringType(), True)
    , StructField('age', IntegerType(), True)
    , StructField('tenure', IntegerType(), True)
    , StructField('balance', DoubleType(), True)
    , StructField('product_count', IntegerType(), True)
    , StructField('has_creditcard', IntegerType(), True)
    , StructField('active_member', IntegerType(), True)
    , StructField('estimated_salary', DoubleType(), True)
    , StructField('complain', IntegerType(), True)
    , StructField('satisfaction_score', IntegerType(), True)
    , StructField('card_type', StringType(), True)
    , StructField('points_earned', IntegerType(), True)
    , StructField('churn', IntegerType(), True)
])

data = spark.read.csv(
    '/FileStore/tables/pyspark_churn.csv'
    , header=True
    , schema=my_schema
)

data.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- credit_score: integer (nullable = true)
 |-- geography: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- balance: double (nullable = true)
 |-- product_count: integer (nullable = true)
 |-- has_creditcard: integer (nullable = true)
 |-- active_member: integer (nullable = true)
 |-- estimated_salary: double (nullable = true)
 |-- complain: integer (nullable = true)
 |-- satisfaction_score: integer (nullable = true)
 |-- card_type: string (nullable = true)
 |-- points_earned: integer (nullable = true)
 |-- churn: integer (nullable = true)



During a testing phase, many errors were thrown from the ml objects due to a lack of data. Further cleaning shows that many columns held no data. The columns were either entirely null or contained only a single value. These are removed so the ml objects can accept meaningful data. Nulls are also dropped and the data is split into a train-test split. From a prior EDA document, a third category `unlabelled_data` is also created as this accounts for 5/13 of the data.

In [0]:
kept_columns = []
for c in data.columns:
    if data.select(c).distinct().count() <= 1 or data.select(c).na.drop().count() == 0:
        print(f'Column {c} has no data.')
    else:
        kept_columns.append(c)
label = 'churn'
raw_features = [c for c in kept_columns if c not in {'customer_id', 'surname', label}]
ml_columns = raw_features + [label]
print(f'''
The left over columns are features
{raw_features}
and label {[label]}.
''')

Column age has no data.
Column has_creditcard has no data.
Column active_member has no data.
Column complain has no data.
Column satisfaction_score has no data.
Column card_type has no data.
Column points_earned has no data.

The left over columns are features
['credit_score', 'geography', 'gender', 'tenure', 'balance', 'product_count', 'estimated_salary']
and label ['churn'].



In [0]:
nonnull_count = data.select(ml_columns).na.drop().count()
if nonnull_count < 1000:
    print(f'There are {nonnull_count} data points after dropping nulls. An imputer is required.')
else:
    data = data.na.drop(how='all')
    print('Nulls have been dropped.')

Nulls have been dropped.


In [0]:
labelled_data = data.select(ml_columns).filter(data.churn.isNotNull())
unlabelled_data = data.select(ml_columns).filter(data.churn.isNull())
print(f'Total customers: {data.count()}')
print(f'Tracked customers: {labelled_data.count()}')
print(f'New customers: {unlabelled_data.count()}')
train, test = labelled_data.randomSplit([0.8,0.2])

Total customers: 13545
Tracked customers: 8430
New customers: 5115


A final check shows that each columns has at least 2 distinct values.

In [0]:
for c in train.columns:
    population = train.select(c).distinct().count()
    assert population>1, f'The ml models will not work on this data: column {c} is invalid.'
    print(f'Column {c} has {population} distinct values.')

Column credit_score has 365 distinct values.
Column geography has 3 distinct values.
Column gender has 2 distinct values.
Column tenure has 11 distinct values.
Column balance has 3240 distinct values.
Column product_count has 4 distinct values.
Column estimated_salary has 6276 distinct values.
Column churn has 2 distinct values.


In [0]:
train.printSchema()

root
 |-- credit_score: integer (nullable = true)
 |-- geography: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- tenure: integer (nullable = true)
 |-- balance: double (nullable = true)
 |-- product_count: integer (nullable = true)
 |-- estimated_salary: double (nullable = true)
 |-- churn: integer (nullable = true)



Columns `geography` and `gender` are strings which will not be accepted by ml objects. This issue can be swiftly fixed by forcing a numerical value on them by way of an indexer. The result is one-hot-encoded to reflect that this is categorical data.

In [0]:
categorical_columns = ['geography', 'gender']
feature_columns = [c if c not in categorical_columns else c+'_onehotencode' for c in raw_features]

geography_indexer = StringIndexer(inputCol='geography',outputCol='geography_index')
geography_encoder = OneHotEncoder(inputCol='geography_index',outputCol='geography_onehotencode')
gender_indexer = StringIndexer(inputCol='gender',outputCol='gender_index')
gender_encoder = OneHotEncoder(inputCol='gender_index',outputCol='gender_onehotencode')
assembler = VectorAssembler(inputCols=feature_columns,outputCol='features')

A dictionary holds the names and objects of the proposed classifiers.

In [0]:
ml_objects = {
    'logistic_regression': LogisticRegression(featuresCol='features',labelCol='churn')
    , 'decision_tree': DecisionTreeClassifier(featuresCol='features',labelCol='churn')
    , 'gbt': GBTClassifier(featuresCol='features',labelCol='churn')
    , 'random_forest': RandomForestClassifier(featuresCol='features',labelCol='churn')
}

Some common sample metrics were selected from the confusion matrix in order to compare the different models with each other.

In [0]:
auc_eval = BinaryClassificationEvaluator(
    rawPredictionCol='prediction'
    , labelCol='churn'
)
acc_eval = MulticlassClassificationEvaluator(
    labelCol='churn'
    , predictionCol='prediction'
    , metricName='accuracy'
)
precision_eval = MulticlassClassificationEvaluator(
    labelCol='churn'
    , predictionCol='prediction'
    , metricName='weightedPrecision'
)
recal_eval = MulticlassClassificationEvaluator(
    labelCol='churn'
    , predictionCol='prediction'
    , metricName='weightedRecall'
)

Everything is put together at the end: creating each ml pipeline, feeding it the training data, and evaluating the result.

In [0]:
saved_models = dict()
for name, ml_obj in ml_objects.items():
    pipeline = Pipeline(stages=[
        geography_indexer
        , gender_indexer
        , geography_encoder
        , gender_encoder
        , assembler
        , ml_obj
    ])
    model = pipeline.fit(train)
    saved_models[name] = model
    result = model.transform(test)
    auc = auc_eval.evaluate(result)
    accuracy = acc_eval.evaluate(result)
    precision = precision_eval.evaluate(result)
    recall = recal_eval.evaluate(result)
    print(f'The {name} model has...')
    print(f'AUC: {auc}')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('')

The logistic_regression model has...
AUC: 0.5818180183096818
Accuracy: 0.7910714285714285
Precision: 0.7678548022354738
Recall: 0.7910714285714285

The decision_tree model has...
AUC: 0.689098125266042
Accuracy: 0.8196428571428571
Precision: 0.8055968845025709
Recall: 0.8196428571428571

The gbt model has...
AUC: 0.6813461872815458
Accuracy: 0.8160714285714286
Precision: 0.8010094019612761
Recall: 0.8160714285714286

The random_forest model has...
AUC: 0.5492275040918002
Accuracy: 0.7910714285714285
Precision: 0.8200320512820514
Recall: 0.7910714285714286




The "best" model can be selected and used on new customers. Here, the `decision_tree` classifier is selected because it has the highest accuracy. Optionally, the predictions may be saved to a database of your choosing.

In [0]:
model = saved_models['decision_tree']
predictions = model.transform(unlabelled_data)